<h2></h2>

<h2>Your task is:</h2>
<ul>
  <li>Finish encoder (2 points)</li>
  <li>Add decoder (3 points)</li>
  <li>Replace saving function to visualisation (1 point)</li>
</ul>
<br> Please do not change the number of epoch to incrase quality of generation, possibly you will not have enough time,

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import CIFAR10
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torchvision.utils import save_image
import numpy as np

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = CIFAR10(root='./data', train=True, transform=transform, download=True)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

lion_dataset = []
for img, label in train_dataset:
    if label == 7:  # Лев имеет метку 7 в CIFAR-10
        lion_dataset.append((img, label))

lion_loader = DataLoader(lion_dataset, batch_size=128, shuffle=True)




# Generator
class Generator(nn.Module):
    def __init__(self, latent_dim):
        super(Generator, self).__init__()
        self.latent_dim = latent_dim
        self.main = nn.Sequential(
            nn.ConvTranspose2d(latent_dim, 512, 4, 1, 0, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),

            nn.ConvTranspose2d(#You code here, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(#You code here),
            nn.ReLU(True),

            nn.ConvTranspose2d(256, #You code here, 4, 2, 1, bias=False),
            nn.BatchNorm2d(#You code here),
            nn.ReLU(True),

            nn.ConvTranspose2d(128, #You code here, 4, 2, 1, bias=False),
            nn.BatchNorm2d(#You code here),
            nn.ReLU(True),

            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, z):
        return self.main(z.view(z.size(0), self.latent_dim, 1, 1)).to('cuda')


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            #You code here
        )

    def forward(self, img):
        return self.main(img).to('cuda')

latent_dim = 100
generator = Generator(latent_dim).to('cuda')
discriminator = Discriminator().to('cuda')
criterion = nn.BCELoss()
optimizer_G = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

num_epochs = 5
for epoch in range(num_epochs):
    for i, data in enumerate(train_loader, 0):
        real_imgs, _ = data
        batch_size = real_imgs.size(0)
        real_imgs = real_imgs.to('cuda')

        optimizer_D.zero_grad()
        z = torch.randn(batch_size, latent_dim, 1, 1).to('cuda')
        fake_imgs = generator(z)
        real_labels = torch.full((batch_size,), 1, dtype=torch.float, device='cuda')
        fake_labels = torch.full((batch_size,), 0, dtype=torch.float, device='cuda')
        real_output = discriminator(real_imgs).view(-1)
        fake_output = discriminator(fake_imgs.detach()).view(-1)
        real_loss = criterion(real_output, real_labels)
        fake_loss = criterion(fake_output, fake_labels)
        d_loss = #You code here sum of real and fake loss
        d_loss.backward()
        optimizer_D.step()

        optimizer_G.zero_grad()
        z = torch.randn(batch_size, latent_dim, 1, 1).to('cuda')
        fake_imgs = generator(z)
        output = discriminator(fake_imgs).view(-1)
        g_loss = criterion(output, real_labels)
        g_loss.backward()
        optimizer_G.step()
        with torch.no_grad():
            z = torch.randn(64, latent_dim, 1, 1).to('cuda')
            fake_imgs = generator(z)
            save_image(fake_imgs, f'generated_images/epoch_{epoch+1}.png', normalize=True)


        if i % 100 == 0:
            print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f' % (epoch, num_epochs, i, len(train_loader), d_loss.item(), g_loss.item()))




Files already downloaded and verified
[0/5][0/391] Loss_D: 1.4126 Loss_G: 3.0209
[0/5][100/391] Loss_D: 0.4212 Loss_G: 3.7976
[0/5][200/391] Loss_D: 0.2289 Loss_G: 4.0345
[0/5][300/391] Loss_D: 0.6261 Loss_G: 3.2202
[1/5][0/391] Loss_D: 0.7986 Loss_G: 1.7975
[1/5][100/391] Loss_D: 0.5808 Loss_G: 2.5497
[1/5][200/391] Loss_D: 0.3369 Loss_G: 3.2090
[1/5][300/391] Loss_D: 0.7791 Loss_G: 3.6145
[2/5][0/391] Loss_D: 0.5101 Loss_G: 3.6225
[2/5][100/391] Loss_D: 0.9757 Loss_G: 2.3579
[2/5][200/391] Loss_D: 0.7118 Loss_G: 3.8935
[2/5][300/391] Loss_D: 0.4214 Loss_G: 4.6231
[3/5][0/391] Loss_D: 0.8119 Loss_G: 2.3780
[3/5][100/391] Loss_D: 0.5161 Loss_G: 2.1721
[3/5][200/391] Loss_D: 0.8338 Loss_G: 2.2319
[3/5][300/391] Loss_D: 0.2831 Loss_G: 4.0873
[4/5][0/391] Loss_D: 0.2925 Loss_G: 3.3062
[4/5][100/391] Loss_D: 0.2146 Loss_G: 2.4185
[4/5][200/391] Loss_D: 0.4851 Loss_G: 2.6131
[4/5][300/391] Loss_D: 1.2232 Loss_G: 4.5151
